## Analise da base de dados `Beta` utilizando algoritmos de ML

Neste notebook será analisado o `Beta dataset` utilizando algoritmos de ML para realizar a: 

1. Extração de características;
2. Seleção de características;
3. Classificação dos dados.

### Pontos importantes do dataset

- Frequências estimuladas (total de 40, com diferença de 0.2Hz uma da outra): 8.0; 8.2;...; 15.6; 15.8;
- Taxa de amostragem: 250Hz

### Analisar os "momentos" em que ocorrem evocação do sinal SSVEP
1. Criar o objeto `MNE` a partir dos dados do participante;
2. Aplicar no objeto `MNE` o filtro passa-faixa nos valores de 6 a 18 Hz;
3. Criar cópias do objeto `MNE` com fatias de tempo menores para analisar momentos que ocorrem estimulos ou não (verificar artigo);

> 0.0 - 0.5 segundos e 2.5 - 3.0 segundos, ocorre apenas ruído; <br/>
 0.5 - 2.5 segundos, ocorre sinal SSVEP (com ruídos)
 
4. Com os sinais separados em objetos `MNE`, aplicar a `FFT`, para que seja possível plotar gráficos que contenham (ou não) as informações:
- Os dados devem ser plotados do domínio da frequência (após a transformada de Fourier). O `FFT` pode ser realizado pela biblioteca `scipy.fft`.
- Deve ser observado que as janelas (a) com ruído não aparecerão de fato o sinal SSVEP.

In [44]:
# Imports

import pandas as pd
import sklearn as skl
import matplotlib as plt
import mne
import numpy as np
from scipy.io import loadmat
from sklearn.feature_selection import RFE
from sklearn.svm import SVC, SVR
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
from sklearn.metrics import accuracy_score, precision_score, classification_report

import warnings
warnings.filterwarnings('ignore')

# beta = loadmat(f"../../datasets/beta/S13.mat")
beta = np.load(f"../../datasets/beta/data_beta.npy")

### Extração de características

Uma característica importante de acordo com o artigo base do dataset `Beta` é o *signal-to-noise ratio* (SNR).

> Incluir equação SNR

Ao finalizar essa etapa, será obtido um vetor de características. Essas podem ser:
- `SNR` (obrigatória);
- Maior valor espectral (FFT em vez do compute_psd);
- Média dos valores espectrais (FFT).

Dimensionalidade dos dados será explicada da seguinte forma:
- `40, 4, 64, 750` -> 40 targets, 4 trials, 64 canais, 750 valores 
- `160, 64 (SNR) + 64 (media) + 64 (maior) ...` Resultando em `160, 192` 

In [2]:
# X = beta['data'][0][0][0]
# y = beta['data'][0][0][1][0][0][4][0]

# X_beta = np.array(np.transpose(X, (3, 2, 0, 1)))
# print(X_beta.shape)

# X_beta = X_beta.reshape(X_beta.shape[0] * X_beta.shape[1], X_beta.shape[2], X_beta.shape[3])
# print(X_beta.shape)

# print("\nLabels:")
# print(y.shape)
# print(y)


mne_data = mne.read_epochs("mne_data_beta.fif")
mne_data

Reading /home/vinicius/Documents/RP_SSVEP/ssvep/src/beta/mne_data_beta.fif ...
Isotrak not found
    Found the data of interest:
        t =       0.00 ...    2996.00 ms
        0 CTF compensation matrices available
Not setting metadata
160 matching events found
No baseline correction applied
0 projection items activated


Number of events,160
Events,10.0: 410.2: 410.4: 410.6: 410.8: 411.0: 411.2: 411.4: 411.6: 411.8: 412.0: 412.2: 412.4: 412.600000000000001: 412.8: 413.0: 413.200000000000001: 413.4: 413.600000000000001: 413.8: 414.0: 414.200000000000001: 414.4: 414.600000000000001: 414.8: 415.0: 415.200000000000001: 415.4: 415.600000000000001: 415.8: 48.0: 48.2: 48.4: 48.6: 48.799999999999999: 49.0: 49.2: 49.4: 49.6: 49.8: 4
Time range,0.000 – 2.996 s
Baseline,off


> Agora iremos estimar o ruído de fundo, para calcular posteriormente o `narrow SNR` e o `wide-band SNR`.

In [3]:
# # Estimando o ruído de fundo

# # intervalos de tempo sem estímulo (0 a 0,5 segundos e 2,5 a 3 segundos)
# base_start = 0
# base_end = 125
# rest_start = 625
# rest_end = 750

# # armazena uma lista com as médias de potência para cada canal
# noise_power = []

# # consideramos a primeira amostra (1º target, 1º trial)
# # for channel_data in X_beta[0, :, :]:
# for channel_data in mne_data.get_data()[0, :, :]:
#     fft_result = np.fft.fft(channel_data)
    
#     # densidade espectral de potência (PSD)
#     psd = np.abs(fft_result) ** 2
    
#     # média da potência nos intervalos de tempo sem estímulo
#     base_power = np.mean(psd[base_start:base_end])
#     rest_power = np.mean(psd[rest_start:rest_end])
    
#     # média das duas médias de potência obtidas anteriormente
#     mean_noise_power = (base_power + rest_power) / 2
#     noise_power.append(mean_noise_power)
    
# #média das médias de potência de todos os canais para estimar o ruído de fundo
# estimated_background_noise = np.mean(noise_power)
# estimated_background_noise

In [4]:
# Estimando o ruído de fundo

# intervalos de tempo sem estímulo (0 a 0,5 segundos e 2,5 a 3 segundos)
ini_ruido = mne_data.copy().crop(tmin=0.0, tmax=0.5)
meio_ruido = mne_data.copy().crop(tmin=0.5, tmax=2.5)
fim_ruido = mne_data.copy().crop(tmin=2.5, tmax=3.0)

# consideramos a primeira amostra (1º target, 1º trial)
fft_ini_result = np.fft.fft(ini_ruido)
fft_fim_result = np.fft.fft(fim_ruido)

# densidade espectral de potência (PSD)
psd_ini = np.abs(fft_ini_result) ** 2
psd_fim = np.abs(fft_fim_result) ** 2

# média da potência nos intervalos de tempo sem estímulo
base_power = np.mean(psd_ini, axis=-1)
rest_power = np.mean(psd_fim, axis=-1)

# média das duas médias de potência obtidas anteriormente
mean_noise_power = np.mean([base_power + rest_power])
    
#média das médias de potência de todos os canais para estimar o ruído de fundo
mean_noise_power

28085.601109550964

> Antes de calcular os SNRs, precisamos obter as amplitudes alvo por meio dos dados EEG:

In [5]:
# from scipy.signal import find_peaks

# sr = 250

# # frequências alvo
# target_frequencies = np.arange(8, 16, 0.2)

# # lista para armazenar as amplitudes nas frequências alvo
# target_amplitudes = []

# # for channel_data in X_beta[0, :, :]:
# for channel_data in mne_data.get_data():
#     fft_result = np.fft.fft(channel_data)
#     psd = np.abs(fft_result) ** 2
#     frequencies = np.fft.fftfreq(len(fft_result), 1 / sr)
#     target_amplitudes_trial = []
#     for target_frequency in target_frequencies:
#         # encontrando o índice da frequência alvo no espectro de frequência
#         index = np.argmin(np.abs(frequencies - target_frequency))
#         # amplitude na frequência alvo
#         amplitude = np.sqrt(psd[index])
#         target_amplitudes_trial.append(amplitude)
        
#     target_amplitudes.append(target_amplitudes_trial)
    
# target_amplitudes = np.array(target_amplitudes)
# target_amplitudes.shape

In [6]:
from scipy.signal import find_peaks

sr = 250

# frequências alvo
target_frequencies = np.arange(8, 16, 0.2)

# lista para armazenar as amplitudes nas frequências alvo
target_amplitudes = []

# for channel_data in X_beta[0, :, :]:
for channel_data in mne_data.get_data():
    target = []
    for eletrodo in channel_data:
        # fft_result = np.fft.fft(channel_data)
        fft_result = np.fft.fft(eletrodo)
        psd = np.abs(fft_result) ** 2
        frequencies = np.fft.fftfreq(len(fft_result), 1 / sr)
        target_amplitudes_trial = []
        for target_frequency in target_frequencies:
            # encontrando o índice da frequência alvo no espectro de frequência
            index = np.argmin(np.abs(frequencies - target_frequency))
            # amplitude na frequência alvo
            amplitude = np.sqrt(psd[index])
            target_amplitudes_trial.append(amplitude)
        
        target.append(target_amplitudes_trial)

    target_amplitudes.append(target)
    
target_amplitudes = np.array(target_amplitudes)
target_amplitudes.shape

(160, 64, 40)

Vamos calcular o SNR de "banda estreita". Pode ser observado pela seguinte equação:

$SNR_{banda\ estreita} = 10 \cdot \log_{10}\left(\frac{\text{energia total do espectro}}{\text{média das amplitudes nas frequências vizinhas}}\right)$

Já o SNR de banda larga é definido da seguinte forma:

$SNR_{banda\ larga} = 10 \cdot \log_{10}\left(\frac{\text{energia total do espectro}}{\text{energia total do espectro de amplitude}}\right)$

In [7]:
# forçando (estragando) valor de "estimated_background_noise" para não sobrar valores negativos
estimated_background_noise = 1.
target_amplitudes_adjusted = target_amplitudes - estimated_background_noise

# subtraindo o ruído de fundo das amplitudes
narrow_band_SNR = 10 * np.log10(target_amplitudes_adjusted / estimated_background_noise)
# print(narrow_band_SNR)
print(narrow_band_SNR.shape)

total_power = np.sum(target_amplitudes_adjusted)
wide_band_SNR = 10 * np.log10(target_amplitudes_adjusted / total_power)
# print(wide_band_SNR)
print(wide_band_SNR.shape)


X = np.array([narrow_band_SNR, wide_band_SNR])
X = X.swapaxes(0, 1)
X = X.swapaxes(1, 2)
X = X.reshape(X.shape[0], X.shape[1], X.shape[2] * X.shape[3])
print()
print(X.shape)

(160, 64, 40)
(160, 64, 40)

(160, 64, 80)


Ao final desta etapa, será obtido um vetor de características. Estas podem ser:
- `narrow SNR` (obrigatória);
- `wide-band SNR` (obrigatória);
- Maior valor espectral (FFT);
- Média dos valores espectrais (FFT).

> Dimensionalidade dos dados será explicada da seguinte forma:

`40, 4, 64, 750` -> 40 targets, 4 trials, 64 canais e 750 valores `160, 64 (SNR) + 64 (média) + 64 (maior) ...`
Resultando em `(160, 192)`.

`(160, 64, 750)` = 2 * `(160, 64, 40)` = `(160, 64, 80)` => (reshape) `(160, 64)`

reshape => `(160, 64, 80)`

> (1) **Seleção de características (MANUAL)**

Remover os eletrodos não listados no artigo => `(160, 9, 80)` => reshape `(160, 720)`

> (2) Vetor de característica para a **seleção de características RFE**

`(160, 64, 80)` => reshape => `(160, 5120)`. Após aplicar o RFE, retorna  `(160, N)`

> (3) Aplicar SVM (kernel='linear'), nas duas seleções

In [8]:
# 1 - Seleção de características (MANUAL)

ch = np.load("../../datasets/beta/channels.npy")

best_ch = ['PZ', 'PO3', 'PO5', 'PO4', 'PO6', 'POZ', 'O1', 'OZ', 'O2']
index_remove = np.where(np.isin(ch, best_ch, invert=True))

X_selected_ch = np.delete(X, index_remove, axis=1)
print(X_selected_ch.shape)

X_selected_ch = X_selected_ch.reshape(X_selected_ch.shape[0], X_selected_ch.shape[1] * X_selected_ch.shape[2])
print(X_selected_ch.shape)


(160, 9, 80)
(160, 720)


In [10]:
# 2 - Vetor de característica para a seleção de características RFE

X_reshape = X.reshape(X.shape[0], X.shape[1] * X.shape[2])
print(X_reshape.shape)

(160, 5120)


### Seleção de características e classificação

Como existem diversos eletrodos (canais) que não obtém sinal SSVEP, podemos extrair as características que não contribuem para a classificação dos dados.

Podemos utilizar o método `RFE` (*Recursive Feature Elimination*) aplicando por meio de `sklearn.feature_selection.RFE`, aprimorando o parâmetro `n_features_to_select` até obter o melhor resultado de classificação.

Para a classificação propriamente dita, é considerado o uso do método `SVM`.

In [55]:
X = StandardScaler().fit_transform(X_selected_ch)
labels = np.load("../../datasets/beta/labels_beta.npy")
y = LabelEncoder().fit_transform(labels)
# X_best_ch = X_selected_ch.copy()
# y = np.load("../../datasets/beta/labels_beta.npy")

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
# X_best_ch.shape

svm_model = SVC(kernel='linear').fit(X_train, y_train)
# svm_model = SVC(kernel='linear')

# X_test = X_test.fillna(X_test.mean())
# y_pred = svm_model.predict(X_test)





ValueError: Input X contains NaN.
SVC does not accept missing values encoded as NaN natively. For supervised learning, you might want to consider sklearn.ensemble.HistGradientBoostingClassifier and Regressor which accept missing values encoded as NaNs natively. Alternatively, it is possible to preprocess the data, for instance by using an imputer transformer in a pipeline or drop samples with missing values. See https://scikit-learn.org/stable/modules/impute.html You can find a list of all estimators that handle NaN values at the following page: https://scikit-learn.org/stable/modules/impute.html#estimators-that-handle-nan-values

In [ ]:
# estimator = SVR(kernel='linear')
# selector = RFE(estimator, n_features_to_select=5)
# selector = selector.fit(X_reshape.shape[0], )